In [1]:
import pandas as pd
import numpy as np
# Для чтения excel файла нужно установить
# pip install xlrd==1.2.0

In [68]:
#Считываем эксель файлы
perechen = pd.read_excel(
    "1._Перечень_организаций_финансового_сектора.xlsx",
     sheet_name=1,
     usecols = [
                "ИНН", 
                "Код сектора экономики СНС-2008",
                "Наименование сектора экономики в соответствии с СНС-2008"
               ],dtype=str)
perechen = perechen.rename(
    columns = {
             "ИНН":"INN",
             "Код сектора экономики СНС-2008": "KOD_SNS",
             "Наименование сектора экономики в соответствии с СНС-2008":
             "sector_economy"}).dropna()
perechen = perechen[["INN", 'KOD_SNS', "sector_economy"]]
perechen["OKK_LICENZIA"] = np.nan
perechen_values = perechen.values

micro = pd.read_excel(
                    "list_MFO.xlsx", header=4
                    , usecols = ["Идентификационный номер налогоплательщика"]
                    , dtype=str)
micro = micro.rename(
    columns = {
        "Идентификационный номер налогоплательщика":"INN"})
micro["OKK_LICENZIA"] = "9.22.2"

straxovanie = pd.read_excel("list_ssd.xlsx", header=2
                            , usecols = ["ИНН"], dtype=str)
straxovanie = straxovanie.rename(columns = {"ИНН":"INN"}).dropna()
straxovanie = straxovanie[["INN"]]
straxovanie["OKK_LICENZIA"] = "9.22.6"

lombards = pd.read_excel("list_PS.xlsx", header=2, usecols = ["ИНН"],
                              sheet_name=["Действующие ломбарды"]
                                 , dtype=str)["Действующие ломбарды"]
lombards = lombards.rename(columns = {"ИНН":"INN"}).dropna()
lombards = lombards[["INN"]]
lombards["OKK_LICENZIA"] = "9.22.7"

kpk = pd.read_excel("list_KPK_gov.xlsx", header=1, usecols = ["ИНН"], dtype=str)
kpk = kpk.rename(columns = {"ИНН":"INN"}).dropna()
kpk = kpk[["INN"]]
kpk["OKK_LICENZIA"] = "9.22.8"

#Размечаем ОКК
for i in range(perechen.shape[0]):
    if (perechen_values[i][1] == "S122") | (perechen_values[i][1] == "S121") | \
    (perechen_values[i][1] == "S1251"):
        perechen_values[i][3] = "9.22.1"
    elif perechen_values[i][1] == "S129":
        perechen_values[i][3] = "9.22.3"
    elif (perechen_values[i][1] == "S124"):
        perechen_values[i][3] = "9.22.4"
    elif (perechen_values[i][1] == "S125"):
        perechen_values[i][3] = "Другие"
    elif (perechen_values[i][1] == "S128"):
        perechen_values[i][3] = "9.22.6"
        
#Объединяем в один фрейм       
perechen = pd.DataFrame(perechen_values, columns=perechen.columns)
df_list = [straxovanie, lombards, kpk]
itog = micro.copy()

for i in df_list:
    itog = itog.append(i, ignore_index=True)

#Убираем дупликаты по ИНН    
def take_unique(res, outer):
    boo = ~outer.INN.isin(res.INN)
    return res.append(outer[boo], ignore_index=True)

to_sql = take_unique(itog, perechen)
to_sql = to_sql[["INN", "OKK_LICENZIA"]]
hard_index = to_sql[to_sql.OKK_LICENZIA.isin(["9.22.1", "9.22.2", "9.22.3",
                                 "9.22.4", '9.22.6', '9.22.7', '9.22.8'])].index
to_sql.loc[:, "flag_hard_reestr"] = 0
to_sql.loc[hard_index, "flag_hard_reestr"] = 1

to_sql.to_excel("reestrs.xlsx", index=False)